In [ ]:
# Deep Learning Libraries
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.layers import Conv2D,MaxPooling2D,Flatten
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.models import load_model

In [ ]:
# preparing data for CNN model
X_train = X_train.as_matrix()
X_test = X_test.as_matrix()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

img_rows = 90
img_cols = 19
img_deps = 1

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, img_deps)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, img_deps)

input_shape = (img_rows, img_cols, img_deps)
early_stopping_monitor = EarlyStopping(patience=3)

In [ ]:
# building the model

model = Sequential()

model.add(Conv2D(32,(3,3),padding='same',input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64,(3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

In [ ]:
# compiling the model
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy']) 

# fitting the model
hist = model.fit(X_train, y_train,
          epochs=30,
          verbose=1,
          validation_split=0.2,
          callbacks=[early_stopping_monitor])


In [ ]:
# Model Evaluation on test-set
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
# Model performance visualization
# plotting Accuracy & Loss vs. Epoch

n_epoch = len(hist.history['acc'])

fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('Epochs',fontweight="bold", size=15)
ax1.set_ylabel('Accuracy', color=color,fontweight="bold", size=15)
ax1.plot(range(1,n_epoch+1), hist.history['acc'], color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Loss', color=color,fontweight="bold", size=15)  # we already handled the x-label with ax1
ax2.plot(range(1,n_epoch+1), hist.history['loss'], color=color)
ax2.tick_params(axis='y', labelcolor=color)
ax2.grid(False)
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.savefig('AccLoss.png')
plt.show()

In [ ]:
# moedl summary
print(model.summary())

# saving the model
model.save('CNN_Chip.h5')